In [2]:
# Setup
from huggingface_hub import hf_hub_download
from datasets import load_dataset
import os 
os.chdir("C:\\Users\\WulfN\\.cache\\huggingface\\hub\\")

%config InteractiveShell.ast_node_interactivity = 'all'
# download model
# hf_hub_download(repo_id = 'prithivMLmods/Deep-Fake-Detector-Model', filename = 'config.json') # presumably the config file is needed first
# model.safetensors, # pytorch_model.bin

# Datasets can be found here: https://huggingface.co/datasets?modality=modality:image&sort=trending&search=deepfake

### Load Data
dataset = load_dataset("Whab/deepfake")

# Preview dataset images

In [ ]:
#import ipywidgets as widgets
# from IPython.display import display
# import ipyplot
#from PIL import Image
#import matplotlib.pyplot as plt

# preview_num = 5

# sample_images = dataset['train'][1:preview_num + 1]

# display_images = [
#     widgets.Image(value=sample_images['image'][i].tobytes(), format='png', width=256, height=256) 
#     for i in range(preview_num)
# ]

# display_images
# display(widgets.HBox(display_images))

## Display images vertically
# for i in range(preview_num):
#     display(sample_images['image'][i])

# display(display_images) 
# #widgets.HBox(display_images)
# # hbox = widgets.HBox(display_images)
# # display(hbox)
# #display_images

Function to create batches of image data

In [3]:
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from functools import partial
# import modin as pd

def create_image_batches(data, batch_size):
    
    num_images = data.num_rows # data[0].num_rows may be needed when using starmap
    batch_size = batch_size

    # index value of first image in batch i
    batch_key = list(range(0, num_images, batch_size))

    # initializing result dataframe
    result = pd.DataFrame({'index': [],
                            'image_batch':[]})

    # batching images in groups of 3
    for i in range(len(batch_key)):

        # index of all images in batch
        batch_indices = list(range(batch_key[i], batch_key[i] + batch_size - 1))

        # appending images from batch into list
        batched_images = []
        for j in batch_indices:
            batched_images.append(chunks[0][j]['image'])

        batch = pd.DataFrame({'index': [i],
                            'image_batch': [batched_images]})
        
        # Adding batch to dataframe
        result = pd.concat([result, batch])
            
    return result

# Function to do parallel processes over chunks of dataframe
def parallel_apply(func, args, n_cores=None):
    
    if n_cores is None:
        n_cores = cpu_count()

    # Apply function on partitions given arguments
    with Pool(n_cores) as pool:
        result = pd.concat(pool.starmap(func, args), chunksize=n_cores) # create_image_batches(batch_size=3)
        
    return result

In [ ]:
n_cores = cpu_count()

def multiply_and_add(x, y):
    """
    Multiply x and y, then add y.
    """
    return x * y + y

if __name__ == "__main__":
    # List of argument pairs (tuples) for the function
    argument_pairs = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6)]

    try:
        with Pool(n_cores) as pool:
            results = pool.starmap(multiply_and_add, argument_pairs)
    finally: 
        pool.close()
        pool.join()

    # Output the results
print("Results:", results)

In [7]:
# Need to parallelize this process
n_cores = cpu_count()
# Test data
test_df = dataset['train'].train_test_split(test_size=0.1, shuffle=False)
split_df1 = test_df['test'].shard(num_shards=n_cores, index=0)
#split_df2 = test_df['test'].shard(num_shards=n_cores, index=1)
#split_df1

if __name__ == "__main__":
    # Create arguments for starmap
    
    chunks = [test_df['test'].shard(num_shards=n_cores, index=i) for i in range(n_cores)] 
    args = []

    for i in range(n_cores):
        args.append((chunks[i], 3))
    
    with Pool(n_cores) as pool:
        result = pool.starmap(create_image_batches, args) # chunks may need to be chunks[index] value

pool.close()

result

: 

: 

In [ ]:
# Goal: Map create_image_batches over each chunk with batch_size of 3
# with Pool(n_cores) as pool:
#     chunks = [test_df['test'].shard(num_shards=n_cores, index=i) for i in range(n_cores)] 
#     args = (chunks, 3)

pool = Pool(n_cores)#with Pool(n_cores) as pool:
chunks = [test_df['test'].shard(num_shards=n_cores, index=i) for i in range(n_cores)] 

# Create arguments for starmap
args = []
for i in range(n_cores):
    args.append((chunks[i], 3))


In [ ]:


#df_length = 100

# Rather than pipe(image i), pipe(batch of images i), try batch of 3 images then 2
# predictions = pd.DataFrame()
# for i in range(batches):
#     image_score = pd.DataFrame(pipe(batches[i])) #dataset['train'][:df_length+1]['image'][i]
#     predictions = pd.concat([predictions, pd.concat({i: image_score}, names=['image_num'])])

In [ ]:
from transformers import pipeline
# Consider Fully Sharded Data Parallel on model for processing

batches_test = batches[:100]

# Huggingface model
pipe = pipeline("image-classification", model="prithivMLmods/Deep-Fake-Detector-Model", device=-1); 

# Get predictions over batches of images
predictions_test = parallel_apply(batches_test, 'image_batch', pipe)
predictions_test

In [ ]:
# Save table in repo
import pickle
predictions.to_pickle('huggingface_predictions')

dist'n of fake and real scores

In [ ]:
# dist'n of fake and real scores
import seaborn as sb
sb.displot(kind='hist', data=predictions.loc[lambda x: x.label == 'Fake'], bins=100)
sb.displot(kind='hist', data=predictions.loc[lambda x: x.label == 'Real'], bins=100)

In [ ]:
# AUC score